In [1]:
import pandas as pd
import progressbar
import igraph as ig
import numpy as np

ModuleNotFoundError: No module named 'progressbar'

In [ ]:
def corr_matrix(ret, thresh=0.95, window=250, enddate="2017-01-24", shrinkage=None, index_ret=None, exp_shrinkage_theta=125,detrended=False):
    """Generates correlation matrix for a window that ends on enddate. Correlation can have exponential shrinkage (giving more weights to recent observations.)
    index_ret is used for detrending. If None, will use average return of all assets.
    Will only use assets with more than thresh%% data available in the window"""
    end = list(ret.index).index(enddate) + 1
    start = end - window
    subret = ret.values[start:end]
    if not (index_ret is None):
        end = list(index_ret.index).index(enddate) + 1
        start = end - window
        index_subret = index_ret.values[start:end].flatten()
    eligible = (~np.isnan(subret)).sum(axis=0) >= thresh * window
    subret = subret[:, eligible]
    index_names = ret.columns[eligible]
    # drop whole column when there are less than or equal to
    # thresh number of non-nan entries in the window
    # sub = ret[start:end]
    subret[np.isnan(subret)] = 0
    if detrended:
        r = subret
        if not (index_ret is None):
            I = index_subret
        else:
            I = subret.mean(axis=1)
        n = len(I)
        alpha = (r.sum(axis=0) * (I * I).sum() - I.sum() * r.T.dot(I)) / (n * (I * I).sum() - (I.sum()) ** 2)
        beta = (n * r.T.dot(I) - I.sum() * r.sum(axis=0)) / (n * (I * I).sum() - (I.sum()) ** 2)
        c = r - alpha - np.outer(I, beta)
        # temp = pd.DataFrame(c)
        # temp.index = company_names
        # temp.columns = company_names
        subret = c
    if shrinkage is None:
        corr_mat = pd.DataFrame(np.corrcoef(subret, rowvar=False))
        corr_mat.columns = index_names
        corr_mat.index = index_names
    # elif shrinkage == "LedoitWolf":
    #     cov = ledoit_wolf(subret, assume_centered=True)[0]
    #     std = np.sqrt(np.diagonal(cov))
    #     corr_mat = (cov / std[:, None]).T / std[:, None]
    #     np.fill_diagonal(corr_mat, 1.0)
    #     corr_mat = pd.DataFrame(data=corr_mat, index=subret.columns, columns=subret.columns)
    elif shrinkage == "Exponential":
        stocknames = index_names
        weight_list = np.exp((np.arange(1, window + 1) - window) / exp_shrinkage_theta)
        weight_list = weight_list / weight_list.sum()
        cov = np.cov(subret, rowvar=False, aweights=weight_list)
        cov_diag = np.sqrt(np.diag(cov))
        corr = (cov / cov_diag).T / cov_diag
        corr_mat = pd.DataFrame(corr)
        corr_mat.columns = stocknames
        corr_mat.index = stocknames
    else:
        print("'shrinkage' can only be None or 'Exponential'")
        return None
    # corr_mat.apply(lambda x:1-x**2 if not math.isnan(x) else np.nan)
    return corr_mat

In [ ]:
def all_corr(ret, thresh=0.95, inclusion=pd.DataFrame(), window=250, shrinkage=None,exp_shrinkage_theta=125, detrended=False, store=None):
    """Computes correlations on all dates in the ret dataframe"""
    print("Computing all correlations with window=%s, shrinkage=%s, theta=%s..." % (window, shrinkage, exp_shrinkage_theta))
    if store is None:
        allcorr = {}
    else:
        allcorr = store
    alldates = ret.index
    alldates.sort_values()
    bar = progressbar.ProgressBar(max_value=len(alldates[window:]))
    for d in alldates[window:]:
        if inclusion.empty:
            allcorr[str(d.strftime("%Y-%m-%d"))] = corr_matrix(ret, thresh, window, enddate=d, shrinkage=shrinkage, exp_shrinkage_theta=exp_shrinkage_theta,detrended=detrended)
        else:
            eligible_stocks = list(inclusion[(inclusion['from']<=d) & ((inclusion['thru'].isnull()) | (inclusion['thru']>=d))]['PERMNO'].unique())
            allcorr[str(d.strftime("%Y-%m-%d"))] = \
            corr_matrix(ret[eligible_stocks], thresh, window, enddate=d, shrinkage=shrinkage, exp_shrinkage_theta = exp_shrinkage_theta, detrended=detrended)
        bar+=1
    alldates = np.array(sorted([s[-10:] for s in allcorr.keys()]))
    return allcorr

In [ ]:
def build_graph(corr, method='gower'):
    """Builds igraph graph from correlation matrix."""
    if method == "gower":
        def distance(weight):
            return (2 - 2 * weight) ** 0.5  # gower
    elif method == "power":
        def distance(weight):
            return 1 - weight ** 2  # power
    node_names = corr.columns.values
    g = ig.Graph.Weighted_Adjacency(corr.values.tolist(), mode="UNDIRECTED", attr="weight", loops=False)
    g.vs['name'] = node_names
    g.es['weight+1'] = np.array(g.es['weight']) + 1.0
    g.es['length'] = distance(np.array(g.es['weight']))
    g.es['absweight'] = np.abs(np.array(g.es['weight']))
    return g


In [ ]:
def MST(corrs, method="gower"):
    """Returns a dictionary of Minimum Spanning Tree for each end date and their graphs in a separate dict"""
    trees = {}
    graphs = {}
    print("Creating MSTs...")
    for d in corrs.keys():
        G = build_graph(corrs[d], method)
        graphs[d[-10:]] = G
        T = G.spanning_tree(return_tree=True, weights='length')
        trees[d[-10:]] = T
    return trees, graphs

In [ ]:
def construct_clusters(trees, method='Newman', n_of_clusters=None):
    """input: trees: iGraph trees
              method: 'Newman' or 'ClausetNewman'
        Returns dicts of the clusters as lists and igraph.clustering objects"""
    print("Computing clusters...")
    sorteddates = sorted(trees.keys())
    usabletrees = trees
    ig.arpack_options.maxiter = 500000
    clusters = {}
    IGclusters = {}
    print("Computing clusterings using method=%s, n_of_clusters=%s" % (method, n_of_clusters))
    bar = progressbar.ProgressBar(max_value=len(sorteddates))
    count = 0
    if method == 'Newman':
        for t in sorteddates:
            if len(usabletrees[t].vs) == 0:
                sys.exit("There are no stocks with sufficient data on %s! Exiting now." % t)
            if n_of_clusters is None:
                c = usabletrees[t].community_leading_eigenvector(weights="weight+1")
            else:
                if len(usabletrees[t].vs) < n_of_clusters:
                    print(
                        "On %s, there are only %s available entities but requiring %s clusters. Use %s clusters instead." % (
                            t, str(len(usabletrees[t].vs)), str(n_of_clusters), str(len(usabletrees[t].vs))))
                extra = 0
                length = 0
                while length != n_of_clusters:
                    c = usabletrees[t].community_leading_eigenvector(weights="weight+1", clusters=min(n_of_clusters,
                                                                                                      len(usabletrees[
                                                                                                              t].vs)) + extra)
                    if len(c) == length:
                        break
                    length = len(c)
                    extra = extra + 1
            IGclusters[t] = c
            clusters[t] = list(c)
            for i in range(0, len(c)):
                clusters[t][i] = [usabletrees[t].vs["name"][j] for j in c[i]]
            count = count + 1
            bar.update(count)
        return clusters, IGclusters
    elif method == 'ClausetNewman':
        for t in sorteddates:
            if len(usabletrees[t].vs) == 0:
                sys.exit("There are no stocks with sufficient data on %s! Exiting now." % t)
            if n_of_clusters is None:
                c = usabletrees[t].community_fastgreedy(weights="weight+1").as_clustering()
            else:
                if len(usabletrees[t].vs) < n_of_clusters:
                    print(
                        "On %s, there are only %s available entities but requiring %s clusters. Use %s clusters instead." % (
                            t, str(len(usabletrees[t].vs)), str(n_of_clusters), str(len(usabletrees[t].vs))))
                c = usabletrees[t].community_fastgreedy(weights="weight+1").as_clustering(
                    n=min(n_of_clusters, len(usabletrees[t].vs)))
            clusters[t] = list(c)
            IGclusters[t] = c
            for i in range(0, len(c)):
                clusters[t][i] = [usabletrees[t].vs["name"][j] for j in c[i]]
            count = count + 1
            bar.update(count)
        return clusters, IGclusters
    elif method == 'infomap':
        for t in sorteddates:
            if len(usabletrees[t].vs) == 0:
                sys.exit("There are no stocks with sufficient data on %s! Exiting now." % t)
            if len(usabletrees[t].vs) < n_of_clusters:
                print(
                    "On %s, there are only %s available entities but requiring %s clusters. Use %s clusters instead." % (
                        t, str(len(usabletrees[t].vs)), str(n_of_clusters), str(len(usabletrees[t].vs))))
            c = usabletrees[t].community_infomap(edge_weights="weight+1")
            clusters[t] = list(c)
            IGclusters[t] = c
            for i in range(0, len(c)):
                clusters[t][i] = [usabletrees[t].vs["name"][j] for j in c[i]]
            count = count + 1
            bar.update(count)
        return clusters, IGclusters
    else:
        print("'method' can only be 'Newman' or 'ClausetNewman' or 'infomap'. Your input was '%s'" % method)
        return None

In [ ]:
data = pd.read_excel('/Users/eugene/Google Drive/Columbia Affiliation/Machine Learning for Global Risk/Market Data/IndexData_new.xlsx', sheet_name = 'Valuefrom 2000')

In [ ]:
data.head()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
IndexData = data.set_index('Date', drop = True)

In [ ]:
IndexData=IndexData[754:4761]

In [ ]:
IndexData.head()

In [ ]:
# window = 250: 2014-10-02 --- 2018-10-04
all_correlation = all_corr(IndexData, thresh=0.75, shrinkage='Exponential',detrended = False)

In [ ]:
all_correlation['2014-10-02'].shape

In [ ]:
trees, graphs = MST(all_correlation, "gower")

In [ ]:
length_list = []

import datetime
keys = sorted(trees.keys(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
for i in keys:
    length_list.append(sum(trees[i].es['weight']))

length = pd.DataFrame(data = length_list, index = keys, columns = ['Length'])


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(keys[0::10], length_list[0::10])

plt.show()

In [ ]:
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

k = keys[0::50]
l = length_list[0::50]

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
yearsFmt = mdates.DateFormatter('%Y')

fig, ax = plt.subplots()
ax.plot(k, l)

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
ax.xaxis.set_minor_locator(months)

# round to nearest years...
datemin = np.datetime64(k[0], 'Y')
datemax = np.datetime64(k[-1], 'Y') + np.timedelta64(1, 'Y')
ax.set_xlim(str(datemin), str(datemax))


# format the coords message box
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.format_ydata = l

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()

plt.show()

In [ ]:
clusters, IGclusters = construct_clusters(trees, method='ClausetNewman')

In [ ]:
clusters['2017-01-12']

In [ ]:
n_of_clusters_list = []

for i in keys:
    n_of_clusters_list.append(len(clusters[i]))

n_of_clusters = pd.DataFrame(data = n_of_clusters_list, index = keys, columns = ['Number'])

In [ ]:
n_of_clusters.to_csv('/home/sbl/Documents/number_of_clusters.csv', encoding = 'utf-8')

In [ ]:
length.to_csv('/home/sbl/Documents/length_of_MST.csv', encoding = 'utf-8')